In [97]:
!pip install pymongo # 安裝 pymongo 函式庫

In [ ]:
import base64
from PIL import Image
from io import BytesIO
from pymongo import MongoClient  
import requests
import time

In [98]:

client = MongoClient(host = '127.0.0.1', port = 27017)
db = client['test_db']
coll = db['Collections']
mydata = {'ex_Key': '20200813_test'}
result = coll.insert_one(mydata)

client.list_database_names()

['admin', 'config', 'local', 'test_db']

In [99]:
# 需要先用robot3T創建mongoDB資料庫
# 安裝pymongo 
# 引入pymongo模組裡面的 MongoClient，還記得我們在Robot3T中建立的資料庫嗎? 接下來我們就會使用 MongoClient來對資料庫進行存取
# 透過 MongoClient我們可以定位到MongoDB服務所在的位址，並將它連接到client變數中
# (因為MongoDB資料庫建立在本機端，所以host填入'127.0.0.1'。 Port則是我們在建立資料庫時指定的27017
# 接下來我們要創近一個新的資料庫(db, database)，可以直接用下面的程序來定義。要注意的是，對MongoDB來說，只有當資料庫裏面有資料這個資料庫才會存在
# 所以我們要把資料放到剛才建立的'test_db'資料庫中


In [100]:

url = 'http://10.17.4.132:8080'
index_url = url+'/index'
r = requests.get(index_url)
if r.status_code == requests.codes.ok:
    print("OK")
res = r.text
head = res.find('<img src=')
tail = head+res[head:].find('>')
img_url = url+res[head:tail].split('\"')[1]
print(img_url)



OK
http://10.17.4.132:8080/static/test.jpg


In [101]:
localtime = time.localtime(time.time())
localtime
timestamp = str(localtime[1])+str(localtime[2])+'_'+str(localtime[3])+str(localtime[4])
timestamp

'813_188'

In [102]:
img_name = timestamp+'-picture.jpg'

html = requests.get(img_url)
with open(img_name, 'wb') as file:
    file.write(html.content)
    

In [103]:
import base64
with open(img_name, "rb") as imageFile:
    strpic = base64.b64encode(imageFile.read()).decode('utf-8')
    coll = db['Collections']
    mydata = {'jpg_base64': strpic}
    result = coll.insert_one(mydata)
    print(result.inserted_id)

5f35110b2f263421aa55592d


In [104]:
with open("imageList.txt", "a+") as file:
    old = file.read() # read everything in the file
    file.seek(0) # rewind
    content = '%\n'+img_name+'\n'+str(result.inserted_id)
    file.write(content+"\n" + old) # write the new line before

In [114]:
# 尋找指定ID的圖片
coll = db['Collections']
img_base64 = []
for i in coll.find():
#     print(str(i['_id']))
#     print(type(i['_id']))
    if str(i['_id']) == '5f35110b2f263421aa55592d':
#         print()
        img_base64.append(i['jpg_base64'])


In [119]:


im = Image.open(BytesIO(base64.b64decode(img_base64[0])))
im.save('mongoDB_out_image.jpg', 'JPEG')